In [1]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
# Specify the actual folder and filenames
folder_path = 'C:/Users/qyu38/Code/RNN_prof_est'  # Replace with your actual folder path
# X_filename = '/X_standardized.npy'
# y_filename = '/y.npy'

X_filename = '/X_standardized_north_all_real.npy'
y_filename = '/y_north_all_real.npy'

X_standardized = np.load(folder_path + X_filename)
y = np.load(folder_path + y_filename)


In [4]:
sample=X_standardized[1]
sample2=X_standardized[-1]
print(sample)



[[-1.66563489e-01  0.00000000e+00 -9.76170444e-01 ... -6.20594222e-04
   2.90077653e-01  6.11379036e-01]
 [-1.66563489e-01  0.00000000e+00 -9.76170444e-01 ... -1.65759146e-02
   2.90077653e-01  5.16529482e-01]
 [-1.66563489e-01  0.00000000e+00 -9.76170444e-01 ... -5.45734233e-02
   2.90077653e-01  4.42722825e-01]
 ...
 [-1.66563489e-01  0.00000000e+00 -9.76170444e-01 ...  4.67492965e-01
   3.78872911e-01  3.60072158e-01]
 [-1.66563489e-01  0.00000000e+00 -9.76170444e-01 ...  2.01828941e-01
   3.78872911e-01  8.47436589e-01]
 [-1.66563489e-01  0.00000000e+00 -9.76170444e-01 ... -9.99449365e-02
   3.78872911e-01  5.71808047e-01]]


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

directory_path = r'\\ad.monash.edu\home\User028\qyu38\Documents\Code\Animation of QC suspension in MatLAB\qcar_animation_for_wfh\data_set\test_data'

num_features = 7  # Number of feature columns
sequence_length = 200  # Length of the sequence for each input
processed_features = []
processed_labels = []

# Iterate over the files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".txt"):  # Assuming the files are .txt files
        file_path = os.path.join(directory_path, filename)

        # Load the file
        df = pd.read_csv(file_path, sep='\t')  # or whatever your separator is
        # breakpoint()
        # Process the file
        for i in range(len(df) - sequence_length + 1):
            feature_sequence = df.iloc[i:i + sequence_length, :num_features]
                        # Check if any speed in the sequence is less than 5
            if not (feature_sequence['speed'] < 5).any():
                label = df.iloc[i, -1]
                # breakpoint()
                #we might need to add a condition here to wipe out features sequences that have speed being less than 5m/s 
                processed_features.append(feature_sequence.values)
                processed_labels.append(label)
        print(f"Processed file: {filename}")


Processed file: north_hatch_m1_hatch__short_arm_2022_12_13_21_41_run1_i.txt
Processed file: north_hatch_m1_hatch__short_arm_2022_12_13_22_28_run2_i.txt
Processed file: north_hatch_m1_hatch__short_arm_2022_12_13_22_58_run3_i.txt
Processed file: north_sedan_m1_sedan__short_arm_2023_03_12_14_20_run1_i.txt
Processed file: north_sedan_m1_sedan__short_arm_2023_03_12_14_50_run2_i.txt
Processed file: north_suv_m1_suv__short_arm_2023_03_28_20_19_run1_i.txt
Processed file: north_suv_m1_suv__short_arm_2023_03_28_20_52_run2_i.txt
Processed file: north_ute_m1_ute__short_arm_2023_02_24_08_31_run1_i.txt


In [9]:
X = np.array(processed_features)
y = np.array(processed_labels).reshape(-1, 1)


In [10]:
print(y.shape)

(45920, 1)


In [1]:
import torch
from torch import nn

# Define the LSTM model
class LSTMNet(nn.Module):
    def __init__(self):
        super(LSTMNet, self).__init__()
        self.lstm = nn.LSTM(input_size=2, hidden_size=100, num_layers=4, batch_first=True)
        self.fc_layers = nn.Sequential(
            nn.Tanh(),
            nn.Linear(100, 100),
            nn.Tanh(),
            nn.Linear(100, 100),
            nn.Tanh(),
            nn.Linear(100, 100),
            nn.Tanh(),
            nn.Linear(100, 1)
        )

    def forward(self, x):
        x, _ = self.lstm(x)  # LSTM output is (output, (h_n, c_n)), we only need the output
        x = x[:, -1, :]  # Get the last time step output from LSTM
        out = self.fc_layers(x)
        return out

# Instantiate the model
model = LSTMNet()

# Display the model architecture
print(model)


LSTMNet(
  (lstm): LSTM(2, 100, num_layers=4, batch_first=True)
  (fc_layers): Sequential(
    (0): Tanh()
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Tanh()
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): Tanh()
    (5): Linear(in_features=100, out_features=100, bias=True)
    (6): Tanh()
    (7): Linear(in_features=100, out_features=1, bias=True)
  )
)
